In [1]:
import torch
import numpy as np
import pandas as pd
# from transformers import AutoTokenizer, AutoModelForMaskedLM

/home/ubuntu/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
meta_data_path='/home/ubuntu/TempRec/Recommender/movie_dataset/movies.dat'
df_meta = pd.read_table(meta_data_path, names= ["movieId", "title", "genres"], sep = "::", encoding = "latin1", engine = "python")


In [3]:
df_meta

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [10]:
temp = pd.read_pickle('/home/ubuntu/TempRec/Recommender/DreamRec/data/zhihu/data_statis.df')

In [11]:
temp

,seq_size,item_num
0,10,4838


In [16]:
temp = pd.read_pickle('/home/ubuntu/TempRec/Recommender/DreamRec/data/Arman/train_data.df')

In [18]:
temp

,seq,len_seq,next,genres,seq_genres,target_genre
0,"[1545, 527, 588, 1, 595, 2355, 783, 2294, 1566...",10,48,"Drama|Drama,War|Animation,Children's,Comedy,Mu...","[16, 18, 17, 1, 13, 13, 13, 17, 4, 13]",2
1,"[2735, 1136, 3421, 1394, 1304, 1259, 1079, 235...",10,2081,"Action,Adventure,Comedy|Comedy|Comedy|Comedy|A...","[1, 1, 1, 1, 15, 1, 1, 13, 1, 1]",4
2,"[1201, 2028, 2947, 2366, 1240, 3418, 1214, 370...",10,1954,"Action,Western|Action,Drama,War|Action|Action,...","[8, 16, 8, 8, 9, 8, 11, 9, 8, 8]",16
3,"[348, 515, 1715, 377, 506, 1580, 1921, 1897, 2...",10,2029,"Comedy|Drama|Thriller|Action,Romance,Thriller|...","[1, 16, 3, 8, 16, 8, 9, 16, 8, 4]",1
4,"[6, 733, 474, 1580, 861, 2353, 1722, 1573, 380...",10,349,"Action,Crime,Thriller|Action,Adventure,Thrille...","[5, 8, 8, 12, 8, 8, 2, 3, 2, 3]",12
...,...,...,...,...,...,...
4825,"[1408, 349, 2353, 1608, 780, 3197, 1375, 380, ...",10,1580,"Action,Romance,War|Action,Adventure,Thriller|A...","[2, 12, 3, 8, 18, 8, 8, 8, 8, 16]",12
4826,"[1248, 1252, 1617, 2726, 541, 2186, 1260, 1267...",10,1179,"Crime,Film-Noir,Thriller|Film-Noir,Mystery,Thr...","[3, 7, 3, 7, 9, 3, 3, 3, 14, 5]",5
4827,"[1200, 2662, 2288, 2287, 3175, 1210, 1584, 264...",10,968,"Action,Sci-Fi,Thriller,War|Action,Sci-Fi,War|A...","[3, 9, 8, 3, 12, 18, 16, 8, 9, 8]",9
4828,"[356, 1079, 1148, 3548, 3088, 232, 1136, 1223,...",10,1183,"Comedy,Romance,War|Comedy|Animation,Comedy|Com...","[2, 1, 1, 1, 1, 1, 1, 1, 16, 16]",2


In [17]:
temp.len_seq.unique()

array([10,  5,  8,  6,  4,  7,  9,  3,  1])

In [1]:
import pandas as pd
df = pd.read_table('/home/ubuntu/TempRec/Recommender/movie_dataset/ratings.dat',names= ['userId', 'movieId', 'rating', 'timestamp'], sep = "::", encoding = "latin1", engine = "python")


/home/ubuntu/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [3]:
from tqdm import tqdm
import pandas as pd

class MoviesDataBuilder:
    def __init__(self, rating_path='/home/ubuntu/TempRec/Recommender/movie_dataset/ratings.dat', meta_data_path='/home/ubuntu/TempRec/Recommender/movie_dataset/movies.dat', number_of_users=None):
        self.rating_path = rating_path
        self.meta_data_path = meta_data_path
        self.number_of_users = number_of_users
        
        self.df_rating =pd.read_table(self.rating_path, names= ['userId', 'movieId', 'rating', 'timestamp'], sep = "::", encoding = "latin1", engine = "python")
        self.df_meta = pd.read_table(self.meta_data_path, names= ["movieId", "title", "genres"], sep = "::", encoding = "latin1", engine = "python")
        # assert False, self.df_rating.iloc[0]
        self.df_rating = pd.merge(self.df_rating, self.df_meta, on='movieId')
        
        self.users = self.df_rating.userId.unique()
        self.df_rating = self.df_rating[self.df_rating.rating >= 4.0]
        
    def build_user(self):
        # Build users' preference
        self.users_pref = []
        self.movies_genres = []
        
        if self.number_of_users is not None: self.users = self.users[:self.number_of_users]
        
        for user in tqdm(self.users, desc='Building user preferences'):
            userData = self.df_rating[self.df_rating.userId == user].sort_values('timestamp')[-11:]
            self.users_pref.append(userData.movieId.to_list())
            
            # The last item is the target, so the genres must not include its genre to prevent data leak
            genres = userData.genres.to_list()[:-1]
            genres = [",".join(item.split('|')) for item in genres]
            genres = "|".join(genres)
            self.movies_genres.append(genres)
    
    def build_dataset(self, max_length_size=10):
        features = []
        target = []
        len_seq = []
        meta_data = []
        
        for index, movies_set in tqdm(enumerate(self.users_pref), desc='Building dataset'):
            length = len(movies_set)
            if length < 2:
                continue
            
            meta_data.append(self.movies_genres[index])
            
            feature_vector = movies_set[:-1]
            target.append(movies_set[-1])
            len_seq.append((length-1))
            if max_length_size > (length-1):
                diff = max_length_size - (length-1)
                feature_vector.extend([feature_vector[-1]]*diff)
            
            assert len(feature_vector) == max_length_size, f"All of the rows must include {max_length_size} items"
            features.append(feature_vector)
            
        
        self.dataset = pd.DataFrame({
                                    'seq': features,
                                    'len_seq': len_seq,
                                    'target': target,
                                    'genres': meta_data
                                }).reset_index(drop=True)

        self.dataset.to_pickle('/home/ubuntu/TempRec/Recommender/movie_dataset/processed_dataset.df')



In [4]:
data_builder = MoviesDataBuilder()

In [5]:
data_builder.build_user()

Building user preferences: 100%|██████████| 6040/6040 [00:06<00:00, 974.11it/s]


In [6]:
data_builder.build_dataset()

Building dataset: 6040it [00:00, 714528.17it/s]


In [9]:
data_builder.dataset

,seq,len_seq,target,genres
0,"[1545, 527, 588, 1, 595, 2355, 783, 2294, 1566...",10,48,"Drama|Drama,War|Animation,Children's,Comedy,Mu..."
1,"[163, 1610, 349, 1527, 2353, 1370, 648, 368, 2...",10,1544,"Action,Romance,Thriller|Action,Thriller|Action..."
2,"[2735, 1136, 3421, 1394, 1304, 1259, 1079, 235...",10,2081,"Action,Adventure,Comedy|Comedy|Comedy|Comedy|A..."
3,"[1201, 2028, 2947, 2366, 1240, 3418, 1214, 370...",10,1954,"Action,Western|Action,Drama,War|Action|Action,..."
4,"[348, 515, 1715, 377, 506, 1580, 1921, 1897, 2...",10,2029,"Comedy|Drama|Thriller|Action,Romance,Thriller|..."
...,...,...,...,...
6032,"[3034, 1447, 2725, 1179, 1748, 1200, 1206, 236...",10,1924,"Animation,Children's|Crime|Drama|Crime,Drama,F..."
6033,"[1200, 2662, 2288, 2287, 3175, 1210, 1584, 264...",10,968,"Action,Sci-Fi,Thriller,War|Action,Sci-Fi,War|A..."
6034,"[356, 1079, 1148, 3548, 3088, 232, 1136, 1223,...",10,1183,"Comedy,Romance,War|Comedy|Animation,Comedy|Com..."
6035,"[926, 1237, 1252, 2019, 923, 858, 1148, 1204, ...",10,912,"Drama|Drama|Film-Noir,Mystery,Thriller|Action,..."


In [7]:
data_builder.dataset.iloc[0].seq      

[1545, 527, 588, 1, 595, 2355, 783, 2294, 1566, 1907]

In [8]:
data_builder.dataset.len_seq.value_counts()

len_seq
10    5923
8       34
9       27
7       21
5       12
6       11
4        6
1        2
3        1
Name: count, dtype: int64

## Set tag for each genre

In [10]:
import random
import pandas as pd

In [11]:
# df = pd.read_csv('/home/ubuntu/TempRec/Recommender/movie_dataset/processed_dataset.csv')
df = data_builder.dataset

In [12]:
df.seq.iloc[0]

[1545, 527, 588, 1, 595, 2355, 783, 2294, 1566, 1907]

In [13]:
genres_set = set()

In [15]:
for row in df.to_numpy():
    genres = row[3]
    for sub_genre in genres.split('|'):
        for genre in sub_genre.split(','):
            genres_set.add(genre)

In [16]:
genres_list = list(genres_set)
genres_id = list(zip(range(1, len(genres_list)+1), genres_list))

In [17]:
pd.DataFrame(genres_id, columns=['genre_id', 'genre']).reset_index(drop=True).to_csv('/home/ubuntu/TempRec/Recommender/movie_dataset/genres.csv', index=False)

In [18]:
# movie_meta_data = pd.read_csv('/home/ubuntu/TempRec/Recommender/movie_dataset/movies.csv')
genres_meta_data = pd.read_csv('/home/ubuntu/TempRec/Recommender/movie_dataset/genres.csv')
movie_meta_data = pd.read_table('/home/ubuntu/TempRec/Recommender/movie_dataset/movies.dat', names= ["movieId", "title", "genres"], sep = "::", encoding = "latin1", engine = "python")

In [19]:
genres_meta_data

,genre_id,genre
0,1,Thriller
1,2,Children's
2,3,War
3,4,Musical
4,5,Drama
5,6,Sci-Fi
6,7,Mystery
7,8,Adventure
8,9,Crime
9,10,Film-Noir


In [20]:
df

,seq,len_seq,target,genres
0,"[1545, 527, 588, 1, 595, 2355, 783, 2294, 1566...",10,48,"Drama|Drama,War|Animation,Children's,Comedy,Mu..."
1,"[163, 1610, 349, 1527, 2353, 1370, 648, 368, 2...",10,1544,"Action,Romance,Thriller|Action,Thriller|Action..."
2,"[2735, 1136, 3421, 1394, 1304, 1259, 1079, 235...",10,2081,"Action,Adventure,Comedy|Comedy|Comedy|Comedy|A..."
3,"[1201, 2028, 2947, 2366, 1240, 3418, 1214, 370...",10,1954,"Action,Western|Action,Drama,War|Action|Action,..."
4,"[348, 515, 1715, 377, 506, 1580, 1921, 1897, 2...",10,2029,"Comedy|Drama|Thriller|Action,Romance,Thriller|..."
...,...,...,...,...
6032,"[3034, 1447, 2725, 1179, 1748, 1200, 1206, 236...",10,1924,"Animation,Children's|Crime|Drama|Crime,Drama,F..."
6033,"[1200, 2662, 2288, 2287, 3175, 1210, 1584, 264...",10,968,"Action,Sci-Fi,Thriller,War|Action,Sci-Fi,War|A..."
6034,"[356, 1079, 1148, 3548, 3088, 232, 1136, 1223,...",10,1183,"Comedy,Romance,War|Comedy|Animation,Comedy|Com..."
6035,"[926, 1237, 1252, 2019, 923, 858, 1148, 1204, ...",10,912,"Drama|Drama|Film-Noir,Mystery,Thriller|Action,..."


In [21]:
def encode_genres(row):
    max_length_size = 10
    genres = row['genres']
    genres_list = list()
    for sub_genre in genres.split('|'):
        genre = random.choice(sub_genre.split(','))
        genre_id = genres_meta_data[genres_meta_data.genre == genre].genre_id.values[0]
        genres_list.append(int(genre_id))
    
    if max_length_size > len(genres_list):
        diff = max_length_size - len(genres_list)
        genres_list.extend([genres_list[-1]]*diff)
        # assert False, (row['seq'], genres, genres_list)
        
    row['seq_genres'] = genres_list
    target_genres = movie_meta_data[movie_meta_data.movieId==row.target].genres.values[0].split('|')
    target_genre = random.choice(target_genres)
    target_genre_id = genres_meta_data[genres_meta_data.genre == target_genre].genre_id.values[0]
    row['target_genre'] = target_genre_id
    return row
    

In [22]:
df = df.apply(encode_genres, axis=1)

In [23]:
df.rename(columns={
    'target':'next', 
}, inplace=True)

In [24]:
df_g = df[['len_seq', 'genres', 'seq_genres', 'target_genre']]

In [25]:
df_g

,len_seq,genres,seq_genres,target_genre
0,10,"Drama|Drama,War|Animation,Children's,Comedy,Mu...","[5, 3, 4, 15, 4, 2, 4, 2, 12, 15]",15
1,10,"Action,Romance,Thriller|Action,Thriller|Action...","[11, 1, 1, 16, 1, 1, 8, 12, 12, 11]",16
2,10,"Action,Adventure,Comedy|Comedy|Comedy|Comedy|A...","[8, 12, 12, 12, 13, 8, 12, 15, 12, 12]",15
3,10,"Action,Western|Action,Drama,War|Action|Action,...","[13, 3, 16, 17, 16, 5, 16, 16, 16, 13]",16
4,10,"Comedy|Drama|Thriller|Action,Romance,Thriller|...","[12, 5, 1, 16, 5, 8, 1, 11, 16, 4]",11
...,...,...,...,...
6032,10,"Animation,Children's|Crime|Drama|Crime,Drama,F...","[15, 9, 5, 9, 10, 6, 6, 6, 6, 6]",6
6033,10,"Action,Sci-Fi,Thriller,War|Action,Sci-Fi,War|A...","[6, 3, 6, 1, 12, 11, 5, 8, 16, 1]",6
6034,10,"Comedy,Romance,War|Comedy|Animation,Comedy|Com...","[12, 12, 15, 5, 12, 5, 12, 12, 11, 5]",3
6035,10,"Drama|Drama|Film-Noir,Mystery,Thriller|Action,...","[5, 5, 7, 16, 5, 9, 15, 8, 10, 8]",11


In [26]:
df

,seq,len_seq,next,genres,seq_genres,target_genre
0,"[1545, 527, 588, 1, 595, 2355, 783, 2294, 1566...",10,48,"Drama|Drama,War|Animation,Children's,Comedy,Mu...","[5, 3, 4, 15, 4, 2, 4, 2, 12, 15]",15
1,"[163, 1610, 349, 1527, 2353, 1370, 648, 368, 2...",10,1544,"Action,Romance,Thriller|Action,Thriller|Action...","[11, 1, 1, 16, 1, 1, 8, 12, 12, 11]",16
2,"[2735, 1136, 3421, 1394, 1304, 1259, 1079, 235...",10,2081,"Action,Adventure,Comedy|Comedy|Comedy|Comedy|A...","[8, 12, 12, 12, 13, 8, 12, 15, 12, 12]",15
3,"[1201, 2028, 2947, 2366, 1240, 3418, 1214, 370...",10,1954,"Action,Western|Action,Drama,War|Action|Action,...","[13, 3, 16, 17, 16, 5, 16, 16, 16, 13]",16
4,"[348, 515, 1715, 377, 506, 1580, 1921, 1897, 2...",10,2029,"Comedy|Drama|Thriller|Action,Romance,Thriller|...","[12, 5, 1, 16, 5, 8, 1, 11, 16, 4]",11
...,...,...,...,...,...,...
6032,"[3034, 1447, 2725, 1179, 1748, 1200, 1206, 236...",10,1924,"Animation,Children's|Crime|Drama|Crime,Drama,F...","[15, 9, 5, 9, 10, 6, 6, 6, 6, 6]",6
6033,"[1200, 2662, 2288, 2287, 3175, 1210, 1584, 264...",10,968,"Action,Sci-Fi,Thriller,War|Action,Sci-Fi,War|A...","[6, 3, 6, 1, 12, 11, 5, 8, 16, 1]",6
6034,"[356, 1079, 1148, 3548, 3088, 232, 1136, 1223,...",10,1183,"Comedy,Romance,War|Comedy|Animation,Comedy|Com...","[12, 12, 15, 5, 12, 5, 12, 12, 11, 5]",3
6035,"[926, 1237, 1252, 2019, 923, 858, 1148, 1204, ...",10,912,"Drama|Drama|Film-Noir,Mystery,Thriller|Action,...","[5, 5, 7, 16, 5, 9, 15, 8, 10, 8]",11


In [128]:
df_g.rename(columns={
    'seq_genres': 'seq',
    'target_genre': 'next'
}, inplace=True)

/tmp/ipykernel_3303086/1643816487.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_g.rename(columns={


In [129]:
df_g.head(5)

,len_seq,genres,seq,next
0,10,"Drama|Drama,War|Animation,Children's,Comedy,Mu...","[16, 18, 17, 1, 13, 13, 13, 17, 4, 13]",2
1,10,"Action,Romance,Thriller|Action,Thriller|Action...","[2, 3, 12, 9, 8, 8, 8, 1, 16, 2]",12
2,10,"Action,Adventure,Comedy|Comedy|Comedy|Comedy|A...","[1, 1, 1, 1, 15, 1, 1, 13, 1, 1]",4
3,10,"Action,Western|Action,Drama,War|Action|Action,...","[8, 16, 8, 8, 9, 8, 11, 9, 8, 8]",16
4,10,"Comedy|Drama|Thriller|Action,Romance,Thriller|...","[1, 16, 3, 8, 16, 8, 9, 16, 8, 4]",1


In [10]:
# data_statis_g = pd.DataFrame({
#     'seq_size': [10],
#     'item_num': [len(genres_meta_data)]
# })

data_statis = pd.DataFrame({
    'seq_size': [10],
    'item_num': 3953
})

In [11]:
# data_statis_g.to_pickle('/home/ubuntu/TempRec/Recommender/DreamRec/data/Arman/data_statis_g.df')
data_statis.to_pickle('/home/ubuntu/TempRec/Recommender/DreamRec/data/Arman/data_statis.df')

In [132]:
test_val_size = int(len(df) * 0.2)

In [133]:
test_val_size

1207

In [134]:
test_val_indices = np.random.choice( len(df), size=test_val_size, replace=False)

In [135]:
test_indices = test_val_indices[:len(test_val_indices)//2]
val_indices = test_val_indices[len(test_val_indices)//2:]

In [136]:
test_val_indices

array([5234, 4289, 2485, ...,  788, 3087, 1848])

In [137]:
train_df = df.drop(test_val_indices).reset_index(drop=True)
test_df = df.iloc[test_indices].reset_index(drop=True)
val_df = df.iloc[val_indices].reset_index(drop=True)

In [138]:
train_df_g = df_g.drop(test_val_indices).reset_index(drop=True)
test_df_g = df_g.iloc[test_indices].reset_index(drop=True)
val_df_g = df_g.iloc[val_indices].reset_index(drop=True)

In [139]:
train_df.to_pickle('/home/ubuntu/TempRec/Recommender/DreamRec/data/Arman/train_data.df')
test_df.to_pickle('/home/ubuntu/TempRec/Recommender/DreamRec/data/Arman/test_data.df')
val_df.to_pickle('/home/ubuntu/TempRec/Recommender/DreamRec/data/Arman/val_data.df')

In [140]:
train_df_g.to_pickle('/home/ubuntu/TempRec/Recommender/DreamRec/data/Arman/train_data_g.df')
test_df_g.to_pickle('/home/ubuntu/TempRec/Recommender/DreamRec/data/Arman/test_data_g.df')
val_df_g.to_pickle('/home/ubuntu/TempRec/Recommender/DreamRec/data/Arman/val_data_g.df')